In [ ]:
# ==========================================
# CELL 1: SETUP CELL - Run this first!
# ==========================================

"""
Pixeldrain to Google Drive Downloader
======================================
A high-speed downloader that saves Pixeldrain files directly to your Google Drive
using Google Colab's infrastructure.

Author: Abhranil Patra
GitHub: https://github.com/NeelPatra/pixeldrain-gdrive-downloader
License: MIT
Version: 1.0.0
"""

import os
import sys
from google.colab import drive

def setup_environment():
    """
    Sets up the Google Drive connection and creates necessary folders.

    This function:
    - Mounts Google Drive to Colab
    - Creates a dedicated download folder
    - Installs aria2c for high-speed downloads

    Returns:
        str: Path to the download folder

    Raises:
        SystemExit: If any setup step fails
    """
    print("=" * 70)
    print("🚀 PIXELDRAIN TO GOOGLE DRIVE DOWNLOADER - ENHANCED VERSION")
    print("=" * 70)
    print("\n📋 Starting setup process...\n")

    # Step 1: Mount Google Drive
    print("📌 [1/3] Connecting to Google Drive...")
    try:
        drive.mount('/content/drive', force_remount=False)
        print("    ✅ Google Drive connected successfully!")
    except Exception as e:
        print(f"    ❌ Error: Failed to mount Google Drive")
        print(f"    📝 Details: {str(e)}")
        print("\n🔧 Troubleshooting:")
        print("    • Make sure you're logged into your Google account")
        print("    • Grant permission when prompted")
        print("    • Try refreshing the page")
        sys.exit(1)

    # Step 2: Create download folder
    download_folder = "/content/drive/MyDrive/Pixeldrain_Downloads"
    print(f"\n📁 [2/3] Setting up download folder...")
    print(f"    📂 Target: {download_folder}")

    try:
        os.makedirs(download_folder, exist_ok=True)
        print("    ✅ Download folder is ready!")
    except PermissionError:
        print("    ❌ Error: Permission denied")
        print("\n🔧 Troubleshooting:")
        print("    • Check your Google Drive permissions")
        print("    • Try disconnecting and reconnecting Drive")
        sys.exit(1)
    except Exception as e:
        print(f"    ❌ Error: Could not create folder")
        print(f"    📝 Details: {str(e)}")
        sys.exit(1)

    # Step 3: Install aria2c
    print("\n⚙️  [3/3] Installing aria2c (High-Speed Download Engine)...")
    print("    ⏳ This may take 10-20 seconds...")

    try:
        result = os.system("apt-get install -y aria2 > /dev/null 2>&1")
        if result == 0:
            print("    ✅ aria2c installed successfully!")
        else:
            raise Exception("Installation returned non-zero exit code")
    except Exception as e:
        print(f"    ❌ Error: Failed to install aria2c")
        print(f"    📝 Details: {str(e)}")
        print("\n🔧 Troubleshooting:")
        print("    • Check your internet connection")
        print("    • Try restarting the runtime: Runtime > Restart runtime")
        sys.exit(1)

    # Setup complete
    print("\n" + "=" * 70)
    print("✅ SETUP COMPLETE! All systems ready.")
    print("=" * 70)
    print(f"\n📂 Your downloads will be saved to:")
    print(f"   {download_folder}")
    print("\n💡 TIP: You can now run the Download cell to start downloading!")
    print("=" * 70 + "\n")

    return download_folder

# Run the setup
try:
    download_folder = setup_environment()
    print("🎯 Setup variable 'download_folder' is now available for the next cell.\n")
except KeyboardInterrupt:
    print("\n\n⚠️  Setup cancelled by user.")
    sys.exit(0)
except Exception as e:
    print(f"\n\n❌ Unexpected error during setup: {str(e)}")
    print("Please report this issue on GitHub if it persists.")
    sys.exit(1)

In [ ]:
# ==========================================
# CELL 2: DOWNLOAD CELL - Run this to download!
# ==========================================

import subprocess
import re
from urllib.parse import urlparse

# ==========================================
# 👇 PASTE YOUR PIXELDRAIN LINK HERE 👇
url = "https://pixeldrain.com/u/XXXXXXXX"
# ==========================================

def validate_pixeldrain_url(url):
    """
    Validates if the provided URL is a valid Pixeldrain link.

    Args:
        url (str): The URL to validate

    Returns:
        bool: True if valid Pixeldrain URL, False otherwise

    Examples:
        >>> validate_pixeldrain_url("https://pixeldrain.com/u/abc123")
        True
        >>> validate_pixeldrain_url("https://google.com")
        False
    """
    # Regex pattern for Pixeldrain URLs
    # Matches: https://pixeldrain.com/u/XXXXXXXX or http://pixeldrain.com/u/XXXXXXXX
    pattern = r'^https?://(www\.)?pixeldrain\.com/u/[a-zA-Z0-9]+$'
    return bool(re.match(pattern, url.strip()))

def convert_to_direct_link(url):
    """
    Converts a Pixeldrain web URL to a direct API download link.

    The API link is faster and more reliable for downloads.

    Args:
        url (str): Original Pixeldrain URL (e.g., https://pixeldrain.com/u/abc123)

    Returns:
        str: Direct download URL (e.g., https://pixeldrain.com/api/file/abc123)

    Examples:
        >>> convert_to_direct_link("https://pixeldrain.com/u/abc123")
        "https://pixeldrain.com/api/file/abc123"
    """
    if "pixeldrain.com/u/" in url:
        return url.replace("pixeldrain.com/u/", "pixeldrain.com/api/file/")
    return url

def download_file(url, output_path):
    """
    Downloads a file using aria2c with optimized settings for Google Drive.

    aria2c is a lightweight multi-protocol & multi-source command-line download utility.
    It supports segmented downloading which significantly speeds up the process.

    Args:
        url (str): Direct download URL
        output_path (str): Destination folder path

    Returns:
        bool: True if download successful, False otherwise

    Technical Details:
        -x16: Use 16 parallel connections
        -s16: Split file into 16 segments
        -k1M: Use 1MB chunks for each segment
        --file-allocation=none: Don't pre-allocate space (faster for cloud storage)
        --summary-interval=5: Show progress every 5 seconds
    """
    cmd = [
        "aria2c",
        "-x", "16",                      # Max connections per server
        "-s", "16",                      # Split into 16 segments
        "-k", "1M",                      # 1MB chunk size
        "--file-allocation=none",        # Optimized for cloud storage
        "--console-log-level=warn",      # Only show warnings/errors
        "--summary-interval=5",          # Progress update interval
        "--download-result=hide",        # Hide detailed results
        "--auto-file-renaming=false",    # Don't auto-rename if file exists
        "-d", output_path,               # Output directory
        url                              # Download URL
    ]

    print("🚀 Starting high-speed download with aria2c...")
    print("📊 Download progress:\n")

    try:
        # Run the download command
        process = subprocess.run(cmd, check=True)
        return process.returncode == 0
    except subprocess.CalledProcessError as e:
        print(f"\n❌ Download failed with error code: {e.returncode}")
        return False
    except FileNotFoundError:
        print("\n❌ Error: aria2c not found!")
        print("🔧 Please run the Setup cell first.")
        return False
    except KeyboardInterrupt:
        print("\n\n⚠️  Download cancelled by user.")
        return False
    except Exception as e:
        print(f"\n❌ Unexpected error: {str(e)}")
        return False

def get_file_id_from_url(url):
    """
    Extracts the file ID from a Pixeldrain URL.

    Args:
        url (str): Pixeldrain URL

    Returns:
        str: File ID or empty string if not found
    """
    match = re.search(r'/u/([a-zA-Z0-9]+)', url)
    return match.group(1) if match else ""

def main(url, download_folder):
    """
    Main orchestration function for the download process.

    This function coordinates all the steps:
    1. Validate the URL
    2. Convert to direct link
    3. Download the file
    4. Report results

    Args:
        url (str): Pixeldrain URL to download
        download_folder (str): Destination folder for downloads
    """
    print("\n" + "=" * 70)
    print("📥 PIXELDRAIN DOWNLOAD PROCESS")
    print("=" * 70 + "\n")

    # Step 1: Validate URL
    print("🔍 [Step 1/3] Validating URL...")
    print(f"    🔗 URL: {url[:50]}{'...' if len(url) > 50 else ''}")

    if not validate_pixeldrain_url(url):
        print("    ❌ Invalid Pixeldrain URL detected!")
        print("\n📝 URL Requirements:")
        print("    • Must start with: https://pixeldrain.com/u/")
        print("    • Example: https://pixeldrain.com/u/abc123xyz")
        print("\n💡 TIP: Copy the full URL from Pixeldrain's website")
        return

    file_id = get_file_id_from_url(url)
    print(f"    ✅ Valid Pixeldrain URL!")
    print(f"    🆔 File ID: {file_id}\n")

    # Step 2: Convert to direct link
    print("🔧 [Step 2/3] Converting to direct download link...")
    direct_url = convert_to_direct_link(url)
    print(f"    ✅ Conversion complete!")
    print(f"    🔗 Direct API link ready\n")

    # Step 3: Download
    print("📥 [Step 3/3] Downloading file...")
    print(f"    📂 Destination: {download_folder}\n")

    success = download_file(direct_url, download_folder)

    # Final Results
    print("\n" + "=" * 70)
    if success:
        print("✅ DOWNLOAD COMPLETE!")
        print("=" * 70)
        print(f"\n📂 File Location:")
        print(f"   {download_folder}")
        print(f"\n📱 How to access your file:")
        print(f"   1. Open Google Drive (drive.google.com)")
        print(f"   2. Navigate to: My Drive > Pixeldrain_Downloads")
        print(f"   3. Your file will be there!")
        print(f"\n💡 PRO TIP: You can now download it to your device from Google Drive")
    else:
        print("❌ DOWNLOAD FAILED!")
        print("=" * 70)
        print("\n🔧 Troubleshooting Steps:")
        print("   1. ✓ Check if the Pixeldrain link is still active")
        print("   2. ✓ Verify you have enough Google Drive storage space")
        print("   3. ✓ Try running the Setup cell again")
        print("   4. ✓ Test with a different Pixeldrain link")
        print("   5. ✓ Check your internet connection")
        print("\n📞 Still stuck? Report the issue on GitHub!")
    print("=" * 70 + "\n")

# ==========================================
# 🎬 EXECUTION STARTS HERE
# ==========================================

# Check if download_folder variable exists (from setup cell)
try:
    download_folder
except NameError:
    print("\n" + "=" * 70)
    print("⚠️  ERROR: Setup not completed!")
    print("=" * 70)
    print("\n❌ The variable 'download_folder' is not defined.")
    print("\n📝 Solution:")
    print("   Please run the SETUP CELL first (the cell above this one)")
    print("   Then come back and run this cell again.")
    print("\n" + "=" * 70 + "\n")
else:
    # Everything is ready, start the download
    main(url, download_folder)